Considere o dataset airports.csv para realizar as seguintes tarefas:

# STARTUP

In [2]:
# Installing required packages
!pip install pyspark
!pip install findspark

You should consider upgrading via the '/opt/python/envs/default/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/python/envs/default/bin/python -m pip install --upgrade pip' command.


In [3]:
import findspark
findspark.init()

In [4]:
import re
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

In [5]:
# Expressoes regulares comuns, pode-ser melhoradda
REGEX_ALPHA    = r'[:aplha:]+'
REGEX_INTEGER  = r'[:digit:]+'
REGEX_FLOAT    = r'[:digit:]+\.[:digit:]+'
REGEX_ALPHANUM = r'[:alnum:]+'
REGEX_EMPTY_STR= r'[:space:]+$'
REGEX_SPECIAL  = r'[:punct:]+'
REGEX_NNUMBER  = r'^N[1-9][0-9]{2,3}([ABCDEFGHJKLMNPRSTUVXWYZ]{1,2})'
REGEX_NNUMBER_INVALID = r'(N0.*$)|(.*[IO].*)'
#no inicio da linha pode ser [0-1]?[0-9])|(2[0-3]), no final ([0-5][0-9])$
REGEX_TIME_FMT = r'^(([0-1]?[0-9])|(2[0-3]))([0-5][0-9])$'

In [6]:
# Criar o contexto do spark
sc = SparkContext()

# Instancia do criador de sessao do spark
spark = (SparkSession.builder
                     .master("local[]")
                     .appName("Desafio Transformação - Week 3"))

In [7]:
import re
#helper functions
def check_empty_column(col):
    return (F.col(col).isNull() | (F.col(col) == '') | F.col(col).rlike(REGEX_EMPTY_STR))

def check_column_range(col, from_value, to_value ):#, leftInclusive = False, rightIncluse = False):
    check_range_expression = "(" + str(F.col(col)) + (" >= " if leftInclusive else " > ") + str(from_value) + " and " \
                                 + str(F.col(col)) + (" <= " if rightIncluse  else " < ") +")" 
    return eval(check_range_expression)
    
def create_regex_from_list(_list):
    return r'|'.join(map(lambda x : f".*({x}).*", _list))

tailnum_chars = F.udf(lambda value: ''.join([c for c in value if not c.isdigit()])[1:])

@F.udf
def get_byregex_and_extract(r_pattern, col_to_search):
    return (getattr(re.search(r_pattern, F.col(col_to_search), re.IGNORECASE), 'groups', lambda:[""])()[0].upper())

getbyreg = F.udf(lambda r_pattern,col_to_search: getattr(re.search(r_pattern, col_to_search, re.IGNORECASE), 'groups', lambda:[u""])()[0].upper())

In [8]:
schema_airports = StructType([
    StructField("faa",  StringType(),  True),
    StructField("name", StringType(),  True),
    StructField("lat",  FloatType(),   True),
    StructField("lon",  FloatType(),   True),
    StructField("alt",  IntegerType(), True),
    StructField("tz",   IntegerType(), True),
    StructField("dst",  StringType(),  True)
])

schema_planes = StructType([
    StructField("tailnum",      StringType(),  True),
    StructField("year",         IntegerType(), True),
    StructField("type",         StringType(),  True),
    StructField("manufacturer", StringType(),  True),
    StructField("model",        StringType(),  True),
    StructField("engines",      IntegerType(), True),
    StructField("seats",        IntegerType(), True),
    StructField("speed",        IntegerType(), True),
    StructField("engine",       StringType(),  True)
])

schema_flights = StructType([
    StructField("year",      IntegerType(), True),
    StructField("month",     IntegerType(), True),
    StructField("day",       IntegerType(), True),
    StructField("dep_time",  StringType(),  True),
    StructField("dep_delay", IntegerType(), True),
    StructField("arr_time",  StringType(),  True),
    StructField("arr_delay", IntegerType(), True),
    StructField("carrier",   StringType(),  True),
    StructField("tailnum",   StringType(),  True),
    StructField("flight",    StringType(),  True),
    StructField("origin",    StringType(),  True),
    StructField("dest",      StringType(),  True),
    StructField("air_time",  IntegerType(), True),
    StructField("distance",  IntegerType(), True),
    StructField("hour",      IntegerType(), True),
    StructField("minute",    IntegerType(), True),
])

In [9]:
df_airports = (spark.getOrCreate().read
               .format('csv')
               .option("inferSchema", "false") 
               .option('header', "true")
               .schema(schema_airports)
               .load('./airports.csv'))
df_planes = (spark.getOrCreate().read
             .format('csv')
             .option('inferSchema', 'false')
             .option('header', 'true')
             .schema(schema_planes)
             .load('./planes.csv'))
df_flights = (spark.getOrCreate().read
              .format('csv')
              .option("inferSchema", "false")
              .option("header", "true")
              .load('./flights.csv'))

In [10]:
df_airports.createOrReplaceTempView('airports_view')
df_flights.createOrReplaceTempView('flights_view')
df_planes.createOrReplaceTempView('planes_view')

In [11]:
#RDDs
rdd_airports   = df_airports.rdd
rdd_df_planes  = df_planes.rdd
rdd_df_flights = df_flights.rdd

In [12]:
#para ver o dataframe
df_airports.show(20)

+---+--------------------+---------+-----------+----+---+---+
|faa|                name|      lat|        lon| alt| tz|dst|
+---+--------------------+---------+-----------+----+---+---+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044| -5|  A|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264| -5|  A|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801| -6|  A|
|06N|     Randall Airport| 41.43191|  -74.39156| 523| -5|  A|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11| -4|  A|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593| -4|  A|
|0G6|Williams County A...|41.467304| -84.506775| 730| -5|  A|
|0G7|Finger Lakes Regi...|42.883564| -76.781235| 492| -5|  A|
|0P2|Shoestring Aviati...|39.794823| -76.647194|1000| -5|  U|
|0S9|Jefferson County ...| 48.05381|-122.810646| 108| -8|  A|
|0W3|Harford County Ai...|39.566837|   -76.2024| 409| -5|  A|
|10C|  Galt Field Airport| 42.40289| -88.375114| 875| -6|  U|
|17G|Port Bucyrus-Craw...|40.781555|  -82.97481|1003| -5|  A|
|19A|Jac

In [13]:
df_airports.rdd.take(20)

[Row(faa='04G', name='Lansdowne Airport', lat=41.13047409057617, lon=-80.61958312988281, alt=1044, tz=-5, dst='A'),
 Row(faa='06A', name='Moton Field Municipal Airport', lat=32.4605712890625, lon=-85.6800308227539, alt=264, tz=-5, dst='A'),
 Row(faa='06C', name='Schaumburg Regional', lat=41.989341735839844, lon=-88.10124206542969, alt=801, tz=-6, dst='A'),
 Row(faa='06N', name='Randall Airport', lat=41.43191146850586, lon=-74.39156341552734, alt=523, tz=-5, dst='A'),
 Row(faa='09J', name='Jekyll Island Airport', lat=31.074472427368164, lon=-81.42778015136719, alt=11, tz=-4, dst='A'),
 Row(faa='0A9', name='Elizabethton Municipal Airport', lat=36.37122344970703, lon=-82.17341613769531, alt=1593, tz=-4, dst='A'),
 Row(faa='0G6', name='Williams County Airport', lat=41.46730422973633, lon=-84.50677490234375, alt=730, tz=-5, dst='A'),
 Row(faa='0G7', name='Finger Lakes Regional Airport', lat=42.88356399536133, lon=-76.78123474121094, alt=492, tz=-5, dst='A'),
 Row(faa='0P2', name='Shoestring

# 1.

In [14]:
df_airports = df_airports.withColumn('alt',(
    F.when(
        F.col('alt') < 0, 0)
     .when(
        check_empty_column('alt'), "M").otherwise(F.col('alt'))
))
df_airports.select('*').where(df_airports.alt ==  0).show()
df_airports.show(20)

+---+--------------------+---------+----------+---+---+---+
|faa|                name|      lat|       lon|alt| tz|dst|
+---+--------------------+---------+----------+---+---+---+
|AGN|Angoon Seaplane Base|57.503613|  -134.585|  0| -9|  A|
|ALZ|Alitak Seaplane Base|56.899445|-154.24777|  0| -9|  A|
|AOS|Amook Bay Seaplan...| 57.47139|-153.81528|  0| -9|  A|
|AQC|Klawock Seaplane ...|55.554657| -133.1017|  0| -9|  A|
|ATT|Camp Mabry Austin...| 30.31666|  -97.7666|  0| -6|  A|
|BYW|Blakely Island Ai...| 48.56025|-122.80243|  0| -8|  A|
|CGA| Craig Seaplane Base| 55.47889|-133.14778|  0| -9|  A|
|CYM|Chatham Seaplane ...|   57.515| -134.9461|  0| -9|  A|
|ELV|Elfin Cove Seapla...| 58.19528| -136.3475|  0| -9|  A|
|EXI|Excursion Inlet S...|58.420555|-135.44917|  0| -9|  A|
|FBS|Friday Harbor Sea...|48.537224|-123.00972|  0| -8|  A|
|FNR|Funter Bay Seapla...|58.254444|-134.89778|  0| -9|  A|
|GAI|Montgomery County...|  39.1006| -77.09576|  0| -5|  A|
|HYG|Hydaburg Seaplane...| 55.20639|-132

In [15]:
#apenas valores 0 na coluna alt
df_airports[df_airports.alt.isin('0')].show()

+---+--------------------+---------+----------+---+---+---+
|faa|                name|      lat|       lon|alt| tz|dst|
+---+--------------------+---------+----------+---+---+---+
|AGN|Angoon Seaplane Base|57.503613|  -134.585|  0| -9|  A|
|ALZ|Alitak Seaplane Base|56.899445|-154.24777|  0| -9|  A|
|AOS|Amook Bay Seaplan...| 57.47139|-153.81528|  0| -9|  A|
|AQC|Klawock Seaplane ...|55.554657| -133.1017|  0| -9|  A|
|ATT|Camp Mabry Austin...| 30.31666|  -97.7666|  0| -6|  A|
|BYW|Blakely Island Ai...| 48.56025|-122.80243|  0| -8|  A|
|CGA| Craig Seaplane Base| 55.47889|-133.14778|  0| -9|  A|
|CYM|Chatham Seaplane ...|   57.515| -134.9461|  0| -9|  A|
|ELV|Elfin Cove Seapla...| 58.19528| -136.3475|  0| -9|  A|
|EXI|Excursion Inlet S...|58.420555|-135.44917|  0| -9|  A|
|FBS|Friday Harbor Sea...|48.537224|-123.00972|  0| -8|  A|
|FNR|Funter Bay Seapla...|58.254444|-134.89778|  0| -9|  A|
|GAI|Montgomery County...|  39.1006| -77.09576|  0| -5|  A|
|HYG|Hydaburg Seaplane...| 55.20639|-132

# 2.

In [16]:
#valor em string A ou horario e data? 
    
df_airports = df_airports.withColumn('dst', (
    F.when(
        (F.col('tz') >= -7) &
        (F.col('tz') <= -5), 'A'
    ).otherwise(F.col('dst'))
))
    
df_airports.show(15)

+---+--------------------+---------+-----------+----+---+---+
|faa|                name|      lat|        lon| alt| tz|dst|
+---+--------------------+---------+-----------+----+---+---+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044| -5|  A|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264| -5|  A|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801| -6|  A|
|06N|     Randall Airport| 41.43191|  -74.39156| 523| -5|  A|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11| -4|  A|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593| -4|  A|
|0G6|Williams County A...|41.467304| -84.506775| 730| -5|  A|
|0G7|Finger Lakes Regi...|42.883564| -76.781235| 492| -5|  A|
|0P2|Shoestring Aviati...|39.794823| -76.647194|1000| -5|  A|
|0S9|Jefferson County ...| 48.05381|-122.810646| 108| -8|  A|
|0W3|Harford County Ai...|39.566837|   -76.2024| 409| -5|  A|
|10C|  Galt Field Airport| 42.40289| -88.375114| 875| -6|  A|
|17G|Port Bucyrus-Craw...|40.781555|  -82.97481|1003| -5|  A|
|19A|Jac

In [17]:
#apenas valores min de -7 e max de -5 na coluna TZ
df_airports[df_airports.tz.isin(-7 ,-5)].show()

+---+--------------------+---------+----------+----+---+---+
|faa|                name|      lat|       lon| alt| tz|dst|
+---+--------------------+---------+----------+----+---+---+
|04G|   Lansdowne Airport|41.130474| -80.61958|1044| -5|  A|
|06A|Moton Field Munic...| 32.46057| -85.68003| 264| -5|  A|
|06N|     Randall Airport| 41.43191| -74.39156| 523| -5|  A|
|0G6|Williams County A...|41.467304|-84.506775| 730| -5|  A|
|0G7|Finger Lakes Regi...|42.883564|-76.781235| 492| -5|  A|
|0P2|Shoestring Aviati...|39.794823|-76.647194|1000| -5|  A|
|0W3|Harford County Ai...|39.566837|  -76.2024| 409| -5|  A|
|17G|Port Bucyrus-Craw...|40.781555| -82.97481|1003| -5|  A|
|1B9| Mansfield Municipal|42.000134| -71.19677| 122| -5|  A|
|1OH|     Fortman Airport|40.555325| -84.38662| 885| -5|  A|
|1RL|Point Roberts Air...| 48.97972|-123.07889|  10| -7|  A|
|24C| Lowell City Airport| 42.95392|  -85.3439| 681| -5|  A|
|24J|Suwannee County A...|30.300125|  -83.0247| 104| -5|  A|
|25D| Forest Lake Airpor

# 3.

In [18]:
#apenas coluna dst com valor U
df_airports[df_airports.dst.isin('U')].show()

+---+--------------------+---------+----------+----+---+---+
|faa|                name|      lat|       lon| alt| tz|dst|
+---+--------------------+---------+----------+----+---+---+
|19A|Jackson County Ai...|34.175865|  -83.5616| 951| -4|  U|
|BLD|Boulder City Muni...|  35.5651|  -114.514|2201| -8|  U|
|GCW|Grand Canyon West...|  35.5925| -113.4859|4825| -8|  U|
|MXY|    McCarthy Airport| 61.43706|-142.90308|1531| -8|  U|
|NGZ|         NAS Alameda|  37.7861| -122.3186|  10| -9|  U|
|SWD|      Seward Airport|60.126938|-149.41881|  22| -8|  U|
|SXQ|    Soldotna Airport|60.474957|-151.03824| 113| -8|  U|
|WWT|      Newtok Airport|60.939167|-164.64111|  25|-10|  U|
+---+--------------------+---------+----------+----+---+---+



In [19]:
df_airports = df_airports.withColumn('dst', (
    F.when(
        (F.col('dst') == 'U'), 'A'
    ).otherwise(F.col('dst'))
))

df_airports.show(15)

+---+--------------------+---------+-----------+----+---+---+
|faa|                name|      lat|        lon| alt| tz|dst|
+---+--------------------+---------+-----------+----+---+---+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044| -5|  A|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264| -5|  A|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801| -6|  A|
|06N|     Randall Airport| 41.43191|  -74.39156| 523| -5|  A|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11| -4|  A|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593| -4|  A|
|0G6|Williams County A...|41.467304| -84.506775| 730| -5|  A|
|0G7|Finger Lakes Regi...|42.883564| -76.781235| 492| -5|  A|
|0P2|Shoestring Aviati...|39.794823| -76.647194|1000| -5|  A|
|0S9|Jefferson County ...| 48.05381|-122.810646| 108| -8|  A|
|0W3|Harford County Ai...|39.566837|   -76.2024| 409| -5|  A|
|10C|  Galt Field Airport| 42.40289| -88.375114| 875| -6|  A|
|17G|Port Bucyrus-Craw...|40.781555|  -82.97481|1003| -5|  A|
|19A|Jac

In [20]:
#apenas coluna dst com valor U
df_airports[df_airports.dst.isin('A')].show()

+---+--------------------+---------+-----------+----+---+---+
|faa|                name|      lat|        lon| alt| tz|dst|
+---+--------------------+---------+-----------+----+---+---+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044| -5|  A|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264| -5|  A|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801| -6|  A|
|06N|     Randall Airport| 41.43191|  -74.39156| 523| -5|  A|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11| -4|  A|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593| -4|  A|
|0G6|Williams County A...|41.467304| -84.506775| 730| -5|  A|
|0G7|Finger Lakes Regi...|42.883564| -76.781235| 492| -5|  A|
|0P2|Shoestring Aviati...|39.794823| -76.647194|1000| -5|  A|
|0S9|Jefferson County ...| 48.05381|-122.810646| 108| -8|  A|
|0W3|Harford County Ai...|39.566837|   -76.2024| 409| -5|  A|
|10C|  Galt Field Airport| 42.40289| -88.375114| 875| -6|  A|
|17G|Port Bucyrus-Craw...|40.781555|  -82.97481|1003| -5|  A|
|19A|Jac

# 4.

In [21]:
#ALASKA: Quando a longitude for menor que -124.
#OFFSHORE: Quando a longitude for maior que -50 ou a latitude for menor que 24.
#MAINLAND-WEST: Quando a longitude for menor ou igual -95 na região dos EUA.
#MAINLAND-EAST: Quando a longitude for maior que -95 na região dos EUA.
#NaN: Caso não atenda nenhuma das condições acima

#check_column_range('lon', -50, 24)

df_airports = df_airports.withColumn('region', (
    F.when(
        F.col('lon') < -124, 'ALASKA'
    ).when(
        ((F.col('lon') > -50) |
         (F.col('lat') < 24)), 'OFFSHORE'
    ).when(
        F.col('lon') <= -95, 'MAINLAND-WEST'
    ).when(
        F.col('lon') >= -95, 'MAINLAND-EAST'
    ).otherwise('NaN')
))

df_airports.show(15)

+---+--------------------+---------+-----------+----+---+---+-------------+
|faa|                name|      lat|        lon| alt| tz|dst|       region|
+---+--------------------+---------+-----------+----+---+---+-------------+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044| -5|  A|MAINLAND-EAST|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264| -5|  A|MAINLAND-EAST|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801| -6|  A|MAINLAND-EAST|
|06N|     Randall Airport| 41.43191|  -74.39156| 523| -5|  A|MAINLAND-EAST|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11| -4|  A|MAINLAND-EAST|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593| -4|  A|MAINLAND-EAST|
|0G6|Williams County A...|41.467304| -84.506775| 730| -5|  A|MAINLAND-EAST|
|0G7|Finger Lakes Regi...|42.883564| -76.781235| 492| -5|  A|MAINLAND-EAST|
|0P2|Shoestring Aviati...|39.794823| -76.647194|1000| -5|  A|MAINLAND-EAST|
|0S9|Jefferson County ...| 48.05381|-122.810646| 108| -8|  A|MAINLAND-WEST|
|0W3|Harford

In [22]:
#Conferindo se todos valores entraram
df_airports.select('region').distinct().show()

+-------------+
|       region|
+-------------+
|       ALASKA|
|     OFFSHORE|
|MAINLAND-EAST|
|MAINLAND-WEST|
+-------------+



# 5. 

In [23]:
#AP: "Airport", "Tradeport", "Heliport", "Airpor", ou "Arpt"
#AD: "Aerodrome"
#AK : "Airpark" ou "Aero Park"
#AS : "Station" ou "Air Station"
#FL : "Field" ou "Fld"
#NaN : Caso não atenda nenhuma das condições acima

# Usando dataframe
AP_TYPES = ["Airport", "Tradeport", "Heliport", "Airpor", "Arpt"]

df_airports = df_airports.withColumn('type',(
        F.when(F.col('name').rlike(create_regex_from_list(AP_TYPES)), 'AP')
         .when(F.col('name').rlike("(AERODROME)"), 'AD')
         .when((
             (F.col('name').rlike(create_regex_from_list(["Airpark", 'Aero Park'])))), 'AS')
         .when(
             (F.col('name').rlike(create_regex_from_list(['Field', "Fld"]))), 'FL'
         ).otherwise('NaN')
    ))

df_airports.show(15)

+---+--------------------+---------+-----------+----+---+---+-------------+----+
|faa|                name|      lat|        lon| alt| tz|dst|       region|type|
+---+--------------------+---------+-----------+----+---+---+-------------+----+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044| -5|  A|MAINLAND-EAST|  AP|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264| -5|  A|MAINLAND-EAST|  AP|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801| -6|  A|MAINLAND-EAST| NaN|
|06N|     Randall Airport| 41.43191|  -74.39156| 523| -5|  A|MAINLAND-EAST|  AP|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11| -4|  A|MAINLAND-EAST|  AP|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593| -4|  A|MAINLAND-EAST|  AP|
|0G6|Williams County A...|41.467304| -84.506775| 730| -5|  A|MAINLAND-EAST|  AP|
|0G7|Finger Lakes Regi...|42.883564| -76.781235| 492| -5|  A|MAINLAND-EAST|  AP|
|0P2|Shoestring Aviati...|39.794823| -76.647194|1000| -5|  A|MAINLAND-EAST| NaN|
|0S9|Jefferson County ...| 4

In [24]:
df_airports.select('Type').distinct().show()

+----+
|Type|
+----+
| NaN|
|  AS|
|  FL|
|  AP|
+----+



# 6.

In [25]:
#True : "Base", "Aaf", "AFs", "Ahp", "Afb", "LRRS", "Lrrs", "Arb", "Naf", "NAS", "Nas", "Jrb", "Ns", "As", "Cgas", "Angb"
#False : Caso nenhuma substrig acima seja identificada

# Usando dataframe
MILITARY_TYPES = ["Base", "Aaf", "AFs", "Ahp", "Afb", "LRRS", "Lrrs", "Arb", "Naf", "NAS", "Nas", "Jrb", "Ns",
"As", "Cgas", "Angb"]

df_airports = df_airports.withColumn('military',(
        F.when(
            F.col('name').rlike(create_regex_from_list(MILITARY_TYPES)), True
        ).otherwise(False)
    ))

df_airports.show(15)

+---+--------------------+---------+-----------+----+---+---+-------------+----+--------+
|faa|                name|      lat|        lon| alt| tz|dst|       region|type|military|
+---+--------------------+---------+-----------+----+---+---+-------------+----+--------+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044| -5|  A|MAINLAND-EAST|  AP|   false|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264| -5|  A|MAINLAND-EAST|  AP|   false|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801| -6|  A|MAINLAND-EAST| NaN|   false|
|06N|     Randall Airport| 41.43191|  -74.39156| 523| -5|  A|MAINLAND-EAST|  AP|   false|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11| -4|  A|MAINLAND-EAST|  AP|   false|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593| -4|  A|MAINLAND-EAST|  AP|   false|
|0G6|Williams County A...|41.467304| -84.506775| 730| -5|  A|MAINLAND-EAST|  AP|   false|
|0G7|Finger Lakes Regi...|42.883564| -76.781235| 492| -5|  A|MAINLAND-EAST|  AP|   false|
|0P2|Shoes

In [26]:
df_airports.select('military').distinct().show()

+--------+
|military|
+--------+
|    true|
|   false|
+--------+



In [1]:
df_airports.select('*').where(df_airports.military == False).show()

NameError: NameError: name 'df_airports' is not defined

In [63]:
#apenas true
df_airports[df_airports.military.isin(True)].show()

+---+--------------------+---------+-----------+----+---+---+-------------+----+--------+--------------+
|faa|                name|      lat|        lon| alt| tz|dst|       region|type|military|administration|
+---+--------------------+---------+-----------+----+---+---+-------------+----+--------+--------------+
|3G4|Ashland County Ai...|40.902973|  -82.25564|1206| -5|  A|MAINLAND-EAST|  AP|    true|             R|
|ADW|         Andrews Afb|38.810806|  -76.86703| 280| -5|  A|MAINLAND-EAST| NaN|    true|           NaN|
|AGN|Angoon Seaplane Base|57.503613|   -134.585|   0| -9|  A|       ALASKA| NaN|    true|           NaN|
|ALZ|Alitak Seaplane Base|56.899445| -154.24777|   0| -9|  A|       ALASKA| NaN|    true|           NaN|
|AOS|Amook Bay Seaplan...| 57.47139| -153.81528|   0| -9|  A|       ALASKA| NaN|    true|           NaN|
|APG|        Phillips Aaf| 39.46622|  -76.16881|  57| -5|  A|MAINLAND-EAST| NaN|    true|           NaN|
|AQC|Klawock Seaplane ...|55.554657|  -133.1017|   0| -

In [64]:
#apenas false
df_airports[df_airports.military.isin(False)].show()

+---+--------------------+---------+-----------+----+---+---+-------------+----+--------+--------------+
|faa|                name|      lat|        lon| alt| tz|dst|       region|type|military|administration|
+---+--------------------+---------+-----------+----+---+---+-------------+----+--------+--------------+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044| -5|  A|MAINLAND-EAST|  AP|   false|           NaN|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264| -5|  A|MAINLAND-EAST|  AP|   false|             M|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801| -6|  A|MAINLAND-EAST| NaN|   false|             R|
|06N|     Randall Airport| 41.43191|  -74.39156| 523| -5|  A|MAINLAND-EAST|  AP|   false|           NaN|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11| -4|  A|MAINLAND-EAST|  AP|   false|           NaN|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593| -4|  A|MAINLAND-EAST|  AP|   false|             M|
|0G6|Williams County A...|41.467304| -84.506775| 730| -

# 7.

In [20]:
#I: "International", "Intl", ou "Intercontinental"
#N: "National", "Natl"
#R : "Regional", "Reigonal", "Rgnl", "County", "Metro" ou "Metropolitan"
#M : "Municipal" "Muni", ou "City"
#NaN : Caso não atenda nenhuma das condições acima

#lists
I_TYPES = ["International", "Intl", "Intercontinental"]
N_TYPES = ["National", "Natl"]
R_TYPES = ["Regional", "Reigonal", "Rgnl", "County", "Metro" "Metropolitan"]
M_TYPES = ["Municipal", "Muni", 'City']

# Usando dataframe
df_airports = df_airports.withColumn('administration',(
        F.when(
            F.col('name').rlike(create_regex_from_list(I_TYPES)), 'I'
        ).when(
            F.col('name').rlike(create_regex_from_list(N_TYPES)), 'N'
        ).when(
            F.col('name').rlike(create_regex_from_list(R_TYPES)), 'R'
        ).when(
            F.col('name').rlike(create_regex_from_list(M_TYPES)), 'M'
        ).otherwise('NaN')
    ))

df_airports.show(15)

+---+--------------------+---------+-----------+----+---+---+-------------+----+--------+--------------+
|faa|                name|      lat|        lon| alt| tz|dst|       region|type|military|administration|
+---+--------------------+---------+-----------+----+---+---+-------------+----+--------+--------------+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044| -5|  A|MAINLAND-EAST|  AP|   false|           NaN|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264| -5|  A|MAINLAND-EAST|  AP|   false|             M|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801| -6|  A|MAINLAND-EAST| NaN|   false|             R|
|06N|     Randall Airport| 41.43191|  -74.39156| 523| -5|  A|MAINLAND-EAST|  AP|   false|           NaN|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11| -4|  A|MAINLAND-EAST|  AP|   false|           NaN|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593| -4|  A|MAINLAND-EAST|  AP|   false|             M|
|0G6|Williams County A...|41.467304| -84.506775| 730| -

In [65]:
#conferindo valores
df_airports.select('administration').distinct().show()

+--------------+
|administration|
+--------------+
|             M|
|             N|
|             R|
|           NaN|
|             I|
+--------------+



In [21]:
# Considere o dataset planes.csv para realizar as seguintes tarefas:

# 1.

In [67]:
data = df_planes.withColumn('tailchar',(
    F.when(check_empty_column('tailnum'), None)
     .when(F.col('tailnum').rlike(REGEX_NNUMBER) ,tailnum_chars(F.col('tailnum')))
) )
data.show(10)

+-------+----+--------------------+------------+--------+-------+-----+-----+---------+----+--------+
|tailnum|year|                type|manufacturer|   model|engines|seats|speed|   engine| age|tailchar|
+-------+----+--------------------+------------+--------+-------+-----+-----+---------+----+--------+
| N906FR|2002|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|  20|      FR|
| N908FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|null|      FR|
| N912FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|null|      FR|
| N918FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|null|      FR|
| N919FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|null|      FR|
| N920FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|null|      FR|
| N921FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo

In [ ]:
#apenas false
df_planes[df_airports.military.isin(False)].show()

# 2.

In [23]:
df_planes = df_planes.withColumn('year', (
    F.when(F.col('year') == 0, 1996)
     .otherwise(F.col('year'))
))

df_planes.show(10)

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N110UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null

# 3.

In [24]:
#Ordene por manufacturer, model e year
#Use o valor da primeira linha anterior que compartilhe os mesmos valores das seguintes colunas, em prioridade:
#A. manufacturer e model
#B. manufacturer

from pyspark.sql.window import Window

year_window = Window.partitionBy().orderBy("manufacturer", "model", "year")

valueColumns = ["manufacturer", "model", "year"]

data = df_planes.select(
    "*",
     *[F.lag(c).over(year_window).alias("old_" + c) for c in valueColumns]
)

df_planes = data.withColumn('year',(
                F.when(
                    (F.col('manufacturer') == F.col('old_manufacturer')) &
                    (F.col('model')        == F.col('old_model')), None
                ).when(
                    (F.col('manufacturer') == F.col('old_manufacturer')), None
                ).otherwise(F.col('year'))
            ))

df_planes.show()

+-------+----+--------------------+------------+--------+-------+-----+-----+---------+----------------+---------+--------+
|tailnum|year|                type|manufacturer|   model|engines|seats|speed|   engine|old_manufacturer|old_model|old_year|
+-------+----+--------------------+------------+--------+-------+-----+-----+---------+----------------+---------+--------+
| N906FR|2002|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|            null|     null|    null|
| N908FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|          AIRBUS| A319-111|    2002|
| N912FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|          AIRBUS| A319-111|    2002|
| N918FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|          AIRBUS| A319-111|    2002|
| N919FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|          AIRBUS| A319-111|    2003|
| N920FR

In [25]:
df_planes.select('*').where(df_planes.year.isNotNull()).show()

+-------+----+--------------------+--------------------+--------------+-------+-----+-----+-------------+--------------------+-------------+--------+
|tailnum|year|                type|        manufacturer|         model|engines|seats|speed|       engine|    old_manufacturer|    old_model|old_year|
+-------+----+--------------------+--------------------+--------------+-------+-----+-----+-------------+--------------------+-------------+--------+
| N906FR|2002|Fixed wing multi ...|              AIRBUS|      A319-111|      2|  147| null|    Turbo-fan|                null|         null|    null|
| N902FR|2001|Fixed wing multi ...|    AIRBUS INDUSTRIE|      A319-111|      2|  147| null|    Turbo-jet|              AIRBUS|     A330-323|    2007|
| N508AA|1975|          Rotorcraft|                BELL|          206B|      1|    5|  112|  Turbo-shaft|       BARKER JACK L|ZODIAC 601HDS|    null|
| N659SW|1985|Fixed wing multi ...|              BOEING|       737-301|      2|  149| null|    Turbo

# 4.

In [26]:
from datetime import date

current_year = date.today().year

df_planes = df_planes.withColumn('age',(
                F.when(
                   check_empty_column('year'), None
                ).otherwise(( current_year - F.col('year')))
            ))

df_planes.show()

+-------+----+--------------------+------------+--------+-------+-----+-----+---------+----------------+---------+--------+----+
|tailnum|year|                type|manufacturer|   model|engines|seats|speed|   engine|old_manufacturer|old_model|old_year| age|
+-------+----+--------------------+------------+--------+-------+-----+-----+---------+----------------+---------+--------+----+
| N906FR|2002|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|            null|     null|    null|  20|
| N908FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|          AIRBUS| A319-111|    2002|null|
| N912FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|          AIRBUS| A319-111|    2002|null|
| N918FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|          AIRBUS| A319-111|    2002|null|
| N919FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|          

In [27]:
df_planes.select('*').where(df_planes.age.isNotNull()).show()

df_planes = df_planes.drop('old_manufacturer', 'old_model', 'old_year')

+-------+----+--------------------+--------------------+--------------+-------+-----+-----+-------------+--------------------+-------------+--------+---+
|tailnum|year|                type|        manufacturer|         model|engines|seats|speed|       engine|    old_manufacturer|    old_model|old_year|age|
+-------+----+--------------------+--------------------+--------------+-------+-----+-----+-------------+--------------------+-------------+--------+---+
| N906FR|2002|Fixed wing multi ...|              AIRBUS|      A319-111|      2|  147| null|    Turbo-fan|                null|         null|    null| 20|
| N902FR|2001|Fixed wing multi ...|    AIRBUS INDUSTRIE|      A319-111|      2|  147| null|    Turbo-jet|              AIRBUS|     A330-323|    2007| 21|
| N508AA|1975|          Rotorcraft|                BELL|          206B|      1|    5|  112|  Turbo-shaft|       BARKER JACK L|ZODIAC 601HDS|    null| 47|
| N659SW|1985|Fixed wing multi ...|              BOEING|       737-301|     

# 5.

In [28]:
#Fixed wing multi engine: MULTI_ENG
#Fixed wing single engine: SINGLE_ENG
#Rotorcraft: ROTORCRAFT

data = df_planes.withColumn('type', (
    F.when(
        check_empty_column('type'), None
    ).when(
        F.col('type').rlike('.*(Fixed wing multi engine).*'), 'MULTI_ENG'
    ).when(
        F.col('type').rlike('.*(Fixed wing single engine).*'), 'SINGLE_ENG'
    ).when(
        F.col('type').rlike('.*(Rotorcraft).*'), 'ROTORCRAFT'
    )
))

data.select('*').where(df_planes.age.isNotNull()).show()

+-------+----+----------+--------------------+--------------+-------+-----+-----+-------------+---+
|tailnum|year|      type|        manufacturer|         model|engines|seats|speed|       engine|age|
+-------+----+----------+--------------------+--------------+-------+-----+-----+-------------+---+
| N906FR|2002|      null|              AIRBUS|      A319-111|      2|  147| null|    Turbo-fan| 20|
| N902FR|2001|      null|    AIRBUS INDUSTRIE|      A319-111|      2|  147| null|    Turbo-jet| 21|
| N508AA|1975|ROTORCRAFT|                BELL|          206B|      1|    5|  112|  Turbo-shaft| 47|
| N659SW|1985|      null|              BOEING|       737-301|      2|  149| null|    Turbo-fan| 37|
| N201AA|1959|      null|              CESSNA|           150|      1|    2|   90|Reciprocating| 63|
| N508JB|2007|      null|  CIRRUS DESIGN CORP|          SR22|      1|    4| null|Reciprocating| 15|
| N563SW|1998|      null|             EMBRAER|       EMB-120|      2|   32| null|   Turbo-prop| 24|


In [29]:
data.show(20)

+-------+----+----+------------+--------+-------+-----+-----+---------+----+
|tailnum|year|type|manufacturer|   model|engines|seats|speed|   engine| age|
+-------+----+----+------------+--------+-------+-----+-----+---------+----+
| N906FR|2002|null|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|  20|
| N908FR|null|null|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|null|
| N912FR|null|null|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|null|
| N918FR|null|null|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|null|
| N919FR|null|null|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|null|
| N920FR|null|null|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|null|
| N921FR|null|null|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|null|
| N922FR|null|null|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|null|
| N923FR|null|null|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|null|
| N924FR|null|null|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|null|

# 6.

In [30]:
#AIRBUS
#BOEING
#BOMBARDIER
#CESSNA
#EMBRAER
#SIKORSKY
#CANADAIR
#PIPER
#MCDONNELL DOUGLAS
#CIRRUS
#BELL
#KILDALL GARY
#LAMBERT RICHARD
#BARKER JACK
#ROBINSON HELICOPTER
#GULFSTREAM
#MARZ BARRY

In [31]:
# Usando dataframe

PLANES_MANUFACTURERS = ["AIRBUS","BOEING","BOMBARDIER","CESSNA","EMBRAER","SIKORSKY","CANADAIR","PIPER","MCDONNELL DOUGLAS","CIRRUS","BELL","KILDALL GARY","LAMBERT RICHARD","BARKER JACK","ROBINSON HELICOPTER","GULFSTREAM","MARZ BARRY"]
REGEX_MANUFACTURERS = r'|'.join(map(lambda x : f".*({x}).*", PLANES_MANUFACTURERS))
"""
def get_byregex_and_extract(r_pattern, col_to_search):
    return (getattr(re.search(r_pattern, F.col(col_to_search), re.IGNORECASE), 'groups', lambda:[""])().upper())

nteste = F.udf(get_byregex_and_extract)

data = df_planes.withColumn('q_manufacturer', (
                F.when(check_empty_column('manufacturer'), 'M')
                 .when(F.col('manufacturer').rlike(REGEX_MANUFACTURERS),
                    nteste(REGEX_MANUFACTURERS, 'manufacturers')
                 )
            ))

data.select('*').where(data.q_manufacturer.isNotNull()).show()"""

data = df_planes.withColumn('manufacturer',(
                F.when(check_empty_column('manufacturer'), 'M')
                 .when(
                     F.col('manufacturer').rlike('.*(AIRBUS).*'),"AIRBUS"
                 )
                 .when(
                     F.col('manufacturer').rlike('.*(BOEING).*'),"BOEING"
                 )
                 .when(
                     F.col('manufacturer').rlike('.*(BOMBARDIER).*'),"BOMBARDIER"
                 )
                 .when(
                     F.col('manufacturer').rlike('.*(CESSNA).*'),"CESSNA"
                 )
                 .when(
                     F.col('manufacturer').rlike('.*(EMBRAER).*'),"EMBRAER"
                 )
                 .when(
                     F.col('manufacturer').rlike('.*(SIKORSKY).*'),"SIKORSKY"
                 )
                 .when(
                     F.col('manufacturer').rlike('.*(CANADAIR).*'),"CANADAIR"
                 )
                 .when(
                     F.col('manufacturer').rlike('.*(PIPER).*'),"PIPER"
                 )
                 .when(
                    F.col('manufacturer').rlike('.*(MCDONNELL DOUGLAS).*'),"MCDONNELL DOUGLAS"
                 )
                 .when(
                     F.col('manufacturer').rlike('.*(CIRRUS).*'),"CIRRUS"
                 )
                 .when(
                     F.col('manufacturer').rlike('.*(BELL).*'),"BELL"
                 )
                 .when(
                     F.col('manufacturer').rlike('.*(KILDALL GARY).*'),"KILDALL GARY"
                 )
                 .when(
                     F.col('manufacturer').rlike('.*(LAMBERT RICHARD).*'),"LAMBERT RICHARD"
                 )
                 .when(
                     F.col('manufacturer').rlike('.*(BARKER JACK).*'),"BARKER JACK"
                 )
                 .when(
                     F.col('manufacturer').rlike('.*(ROBINSON HELICOPTER).*'),"ROBINSON HELICOPTER"
                 )
                 .when(
                     F.col('manufacturer').rlike('.*(GULFSTREAM).*'),"GULFSTREAM"
                 )
                 .when(
                     F.col('manufacturer').rlike('.*(MARZ BARRY).*'),"MARZ BARRY"
                 )
            ))

data.show()

#com rdd
#lambda manufacture:  manufacture, re.IGNORECASE), 'groups', lambda:[u""])()[0].upper()
#rdd_df_planes.map(lambda manufacturer: re.search(REGEX_MANUFACTURERS, manufacturer[3], re.IGNORECASE).group(1)  ).collect()

+-------+----+--------------------+------------+--------+-------+-----+-----+---------+----+
|tailnum|year|                type|manufacturer|   model|engines|seats|speed|   engine| age|
+-------+----+--------------------+------------+--------+-------+-----+-----+---------+----+
| N906FR|2002|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|  20|
| N908FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|null|
| N912FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|null|
| N918FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|null|
| N919FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|null|
| N920FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|null|
| N921FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147| null|Turbo-fan|null|
| N922FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147

# 7.

In [35]:
data = df_planes.withColumn('model', (
            F.when(
                F.col('model').rlike('\(.*\)'), F.regexp_replace('model', r'[(].*[)]', '')

            ).otherwise(F.col('model'))
        ))

#check for values with parentheses
data.select('*').where(data.model.rlike('\(.*\)')).sort(F.desc('model')).show()

parenthesis_sql = spark.getOrCreate().sql(
"""
select *,
        case 
            when model like '\(.*\)' then regexp_replace(model, '[(].*[)]', '')
        else
            model
        end as model

    from planes_view
"""
)

parenthesis_sql.show()

+-------+----+----+------------+-----+-------+-----+-----+------+---+
|tailnum|year|type|manufacturer|model|engines|seats|speed|engine|age|
+-------+----+----+------------+-----+-------+-----+-----+------+---+
+-------+----+----+------------+-----+-------+-----+-----+------+---+

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|   model|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|A320-214|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|A320-214|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|A320-214|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|A320-214|
| N107US|199

# 8.

In [36]:
data = df_planes.withColumn("speed", (F.when(
                                    check_empty_column('speed'), (F.ceil(F.column('seats').cast(FloatType()) /.38))
                                ).otherwise(F.col('speed')))
                           )
data.show()

sql_results = spark.getOrCreate().sql("""
select
        *,
        case
            when 
               speed is null or speed like '' then ceil(seats/.38)
            else
               speed
        end as speed
    from planes_view
""") 

+-------+----+--------------------+------------+--------+-------+-----+-----+---------+----+
|tailnum|year|                type|manufacturer|   model|engines|seats|speed|   engine| age|
+-------+----+--------------------+------------+--------+-------+-----+-----+---------+----+
| N906FR|2002|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147|  387|Turbo-fan|  20|
| N908FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147|  387|Turbo-fan|null|
| N912FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147|  387|Turbo-fan|null|
| N918FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147|  387|Turbo-fan|null|
| N919FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147|  387|Turbo-fan|null|
| N920FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147|  387|Turbo-fan|null|
| N921FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147|  387|Turbo-fan|null|
| N922FR|null|Fixed wing multi ...|      AIRBUS|A319-111|      2|  147

# 9. Crie a coluna engine_type (category) e atribua os valores a partir de substrings identificadas na coluna engine de acordo com as condições abaixo:

In [ ]:
data = df_planes.withColumn('engine_type', (
    F.when(
        check_empty_column('engine'), 'M'
    ).when(
        F.col('engine') == 'Turbo-fan', 'FAN'
    ).when(
        F.col('engine') == 'Turbo-jet', 'JET'
   ).when(
        F.col('engine') == 'Turbo-prop', 'PROP'
   ).when(
        F.col('engine') == 'Turbo-shaft', 'SHAFT'
   ).when(
        F.col('engine') == '4 Cycle', 'Cycle'
   )
))

data.show()

## Flights Dataset - Perguntas

# 1.

In [38]:
data = df_flights.withColumn('hour',(
            F.when(
                check_empty_column('hour'), 0
            ).otherwise(F.col('hour'))
        )).withColumn('minute',(
            F.when(
                check_empty_column('minute'), 0
            ).otherwise(F.col('minute'))
        ))
data.select('*').where(F.col('hour').isNull() | F.col('minute').isNull()).show()

data.show()

df_flights = data

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    

# 2.

In [39]:
data = df_flights.withColumn('hour', (
    F.when(
        (F.col('hour') == 24), 0
    ).otherwise(F.col('hour'))
))

data.select('*').where(F.col('hour').isNull()).show()
data.select('*').where(F.col('hour') > 23).show()


data.show()

df_flights = data

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+

+----+-----+---+--------+---------+---

# 3.

In [ ]:
mapping = F.create_map()
data = df_flights.withColumn('dep_datetime', (
    F.when(
        check_empty_column('minute'), None
    ).otherwise(F.concat(
        F.col('year'), F.lit('-'), F.col('month'),F.lit('-'),F.col('day'),
        F.lit(' '), F.col('hour'), F.lit(':'), F.col('minute'), F.lit(':00')
    ))
))

data.show()

df_flights = data

# 4.

In [40]:
data = df_flights.withColumn('dep_time', (
    F.when(
        check_empty_column('dep_time'), (
                F.concat(
                    F.col('hour'), F.col('minute')
                )
        )
    ).otherwise(F.col('dep_time'))
))

data.show()

df_flights.select('*').where(df_flights.dep_time == None).show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

# 5.

In [42]:
data = df_flights.withColumn('dep_delay', (
    F.when(
        (check_empty_column('dep_delay')), 0
    ).otherwise(F.col('dep_delay'))
))

data.select('*').where(F.col('dep_delay').isNull()).show()


data.show()

df_flights = data

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    

# 6.

In [ ]:
data = df_flights.withColumn('arr_delay', (
    F.when(
        (check_empty_column('arr_delay')), 0
    ).otherwise(F.col('arr_delay'))
))

data.select('*').where(F.col('arr_delay').isNull()).show()


data.show()

df_flights = data

# 7.

In [44]:
df_flights.show(10)

data = df_flights.drop(*('year', 'month', 'day', 'hour', 'minute'))
data.show(10)

df_flights = data

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

# 8. Crie a coluna air_time_projected(int) de acordo com a fórmula [Distance x 0.1+ 20]

In [ ]:
df_flights = df_flights.withColumn('air_time_projected',
                                   ((F.col('distance') * 0.1) + 20).cast(IntegerType()))

In [ ]:
df_flights.printSchema()

In [ ]:
df_flights.groupBy("air_time_projected").count().distinct().orderBy("air_time_projected", ascending=True).show(270)

## Pergunta 9

In [ ]:
df_flights.toPandas()

In [ ]:
df_flights_1 = df_flights.groupBy("origin", "dest").agg(F.avg('air_time').cast(IntegerType()).alias('air_time_expected')).orderBy('origin')

In [ ]:
print('Actual columns:', df_flights_1.columns)


df_flights_1 = (df_flights_1.withColumnRenamed('origin', 'O1')
                          .withColumnRenamed('dest', 'D1')
                          .withColumnRenamed('air_time_expected', 'air_time_expected')
                                       
              )
print('modified columns:', df_flights_1.columns)

df_flights_1.show()

In [ ]:
df_flights_Final = (df_flights.join(df_flights_1,
                  (df_flights.origin == df_flights_1.O1)
                     & (df_flights.dest == df_flights_1.D1)
                        ,"left")
)

In [ ]:
df_flights_Final.toPandas()

In [ ]:
df_flights = df_flights_Final.drop("O1", "D1")

In [ ]:
df_flights.toPandas()

## Pergunta 10

In [ ]:
df_flights.groupBy("air_time").count().distinct().orderBy("air_time", ascending=True).show()

In [ ]:
# Função greatest (Retorna o maior valor de todos os parâmetros)
df_flights = (df_flights.withColumn('air_time',
                                  F.when(F.col('air_time').isNull(), 
                                         F.greatest(F.col('air_time_projected'), F.col('air_time_expected')))
                                   .otherwise(F.col('air_time'))                     
                               
            ))

In [ ]:
df_flights.groupBy("air_time").count().distinct().orderBy("air_time", ascending=True).show()

## Pergunta 11

In [ ]:
df_flights.filter(F.col('arr_time') == 'NA').toPandas()

In [ ]:
# Transforma a coluna dep_datetime em segundos com a função unix_timestamp e soma com a coluna air_time 
# transformada em segundos, transformando novamente em timestamp depois
df_flights = df_flights.withColumn('sum', 
                        (F.unix_timestamp("dep_datetime") + 
                        (F.col('air_time') * 60).cast('int')).cast('timestamp'))

# Verifica aonde a coluna arr_time está nulo e troca o valor
df_flights = df_flights.withColumn('arr_time',
             F.when(F.col('arr_time') == 'NA', F.format_string('%d%d', F.hour(F.col('sum')), 
                                                                       F.minute(F.col('sum'))))
             .otherwise(F.col('arr_time')))

# Remove a coluna sum
df_flights = df_flights.drop('sum')

df_flights.toPandas()

In [ ]:
df_flights.select(F.col('origin'),
                  F.col('dest'),
                  F.col('arr_time'),
                  F.col('dep_time'),
                  F.col('air_time')).where((F.col('origin') == 'SEA') & (F.col('dest') == 'PHX')).show()

In [ ]:
df_flights.filter(F.col('arr_time') == 'NA').show(truncate = 5)

(df_flights
.select(F.col('dep_time'), 
        F.col('dep_delay'), 
        F.col('arr_time'), 
        F.col('arr_delay'), 
        F.col('air_time'), 
        F.col('tailnum'))
.where(F.col('tailnum') == 'N612AS')
.show(50))

## Pergunta 12

In [ ]:
df_flights = df_flights.withColumn('haul_duration', (
                      F.when((F.col('air_time').between(20,180)), "SHORT-HAUL")
                       .when((F.col('air_time').between(181,360)), "MEDIUM-HAUL")
                       .otherwise('LONG-HAUL')))

In [ ]:
df_flights.groupBy("haul_duration").count().distinct().orderBy("haul_duration", ascending=True).show()

## Pergunta 13

In [ ]:
df_flights.groupBy("dep_season").count().distinct().orderBy("dep_season", ascending=True).show()

## Pergunta 14

In [ ]:
df_flights = df_flights.withColumn('dep_delay_category', (
                      F.when((F.col('dep_delay') < 0), "ANTECIPATED")
                       .when((F.col('dep_delay') == 0), "INTIME")
                       .when((F.col('dep_delay') >= 60 ), "MAJOR")
                       .otherwise("MINOR")))

In [ ]:
df_flights.groupBy("dep_delay_category").count().distinct().orderBy("dep_delay_category", ascending=True).show()